In [ ]:
# Creating simple vector and raster test cases

In [ ]:
import geopandas as gpd
import shapely

def get_square_row(num_squares=2, translation=(1,0), square_side_len=1, origin=(0,0)):
    """
    Creates a row of squares with side length `square_side_len`, 
    each square is translated to the previous by `translation`.
    Starts with the bottom left corner of first square at
    `origin`. Returns gpd GeoDataFrame with square polygons.
    """
    
    out_df = gpd.GeoDataFrame(columns=['id', 'geometry', 'area'])
    
    # setting up the coordinates of first square
    square_coords = [(0,0), (1,0), (1,1), (0,1), (0,0)]
    # scaling up by square_side_len
    square_coords = [tuple([coord * square_side_len for coord in coords]) for coords in square_coords]
    # setting to origin coords
    square_coords = [[sum(x) for x in zip(coords,origin)] for coords in square_coords]
    
    square = shapely.geometry.Polygon(square_coords)
    
    # creating the row of squares
    for i in range(num_squares):
        xoff = i*translation[0]
        yoff = i*translation[1]
        tmp_square = shapely.affinity.translate(square, xoff=xoff, yoff=yoff)
        out_df.loc[i] = [i, tmp_square, tmp_square.area]
        
    return out_df
    
test_df = get_square_row(num_squares=5, translation=(1,1), 
                         square_side_len=1, origin=(0,0))

In [ ]:
test_df.plot()

In [ ]:
import geopandas as gpd
from shapely.strtree import STRtree
import networkx as nx

def create_graph(gpd_df, attributes=None):
    """ Creates graph from GeoDataFrame.
    """
    
    # If no attribute list is given, all
    # columns in gpd_df are used
    if not attributes:
        attributes = test_df.columns.tolist()
    
    geom = gpd_df.geometry.tolist()
    # build dict mapping hashable unique object ids for each polygon to their index in geom
    id_dict = {id(poly):index for index, poly in enumerate(geom)}
    # build Rtree from geometry
    tree = STRtree(geom)

    # Combining this with graph
    graph_dict = {}
    G = nx.Graph()
    for index, polygon in enumerate(geom):
        # find the indexes of all polygons which touch the borders of or overlap with this one
        neighbours = [id_dict[id(nbr)] for nbr in tree.query(polygon) if nbr.touches(polygon) or nbr.overlaps(polygon)]
        # this dict maps polygon indices in gpd_df to a list of neighbouring polygon indices
        graph_dict[index] = neighbours
        row = gpd_df.loc[index]
        # getting dict of column values in row
        row_attributes = dict(zip(attributes,[row[attr] for attr in attributes]))
        # add each polygon as a node to the graph with all attributes
        G.add_node(index, **row_attributes)

    # iterate through the dict and add all edges between neighbouring polygons
    for polygon_id, neighbours in graph_dict.items():
        for neighbour_id in neighbours:
            G.add_edge(polygon_id, neighbour_id)
            
    return G
            
graph = create_graph(test_df)

In [ ]:
print(graph.number_of_nodes(), graph.number_of_edges())

In [ ]:
nx.draw(graph)

In [ ]:
graph.nodes[4]